# Exercise 1

# Import all Libraries

In [348]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Read Wikipedia Page to get the table

In [349]:
import requests
#Read the page
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
dfs = pd.read_html(url_wiki, header=0)

neighborhoods = dfs[0].copy()
neighborhoods.head()
#neighborhoods.count()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Remove 'Not assigned' cases

In [350]:
#Remove 'Not assigned' cases

print('No. of Neighborhoodes before processing: ', len(neighborhoods))

dropIndex = neighborhoods[neighborhoods['Borough'] == 'Not assigned'].index
neighborhoods.drop(index = dropIndex, inplace = True)

neighborhoods.Neighbourhood[neighborhoods['Neighbourhood'] == 'Not assigned'] = neighborhoods.Borough

print('No. of Neighborhoodes after processing: ', len(neighborhoods))
neighborhoods.reset_index(inplace=True)
neighborhoods.drop('index', axis = 1, inplace=True)
neighborhoods.head(5)

No. of Neighborhoodes before processing:  289
No. of Neighborhoodes after processing:  212


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Group the Neighbourhoods

In [351]:
#Grouped Neighborhoods
g_neighborhoods = neighborhoods.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()
g_neighborhoods.rename(index=str, columns={"Postcode": "Postal Code", "Neighbourhood": "Neighborhood"}, inplace=True)
g_neighborhoods.head(5)


,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Size of the grouped Neighborhoods

In [352]:
g_neighborhoods.shape

(103, 3)

# Exercise 2 - Merge and display Neighborhood data with latitude and longitude

## Download & Read Geospatial Data

In [353]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data

print('Data downloaded!')

geo_df = pd.read_csv('Geospatial_data.csv')
geo_df.head()

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge and display Neighborhood data with latitude and longitude

In [354]:
df_neighborhoods = pd.merge(g_neighborhoods, geo_df, how='outer', on='Postal Code')
df_neighborhoods.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Exercise 3 - Plot boroughs that contain the word Toronto 

## Use geopy library to get the latitude and longitude values of Toronto

In [355]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with all neighborhoods superimposed on top 

In [356]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neighborhoods['Latitude'], df_neighborhoods['Longitude'], df_neighborhoods['Borough'], df_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
#map_Toronto

## Identify Boroughs that contain the word Toronto 

In [357]:
df_neighborhoods['Borough'].value_counts() #All Neighbourhoods
Toronto_data = df_neighborhoods[df_neighborhoods['Borough'].str.contains('Toronto') == True] #TO be used further
print("Boroughs containing word Toronto with neighberhood  count :\n", Toronto_data['Borough'].value_counts()) #Filtered ones with Toronto

#List of unique Boroughs
Borough_list = Toronto_data['Borough'].unique()
print("Unique Borough List : ", Borough_list)

Boroughs containing word Toronto with neighberhood  count :
 Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64
Unique Borough List :  ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


## Define Foursquare Credentials and Version

In [358]:
CLIENT_ID = '12GJPABT1NNWQIJUMSESEY41PHZV0TPO1H3FRXSNZ3ZK0JMD' # your Foursquare ID
CLIENT_SECRET = 'FV4DBNONPOV1F3K3NPOGKXB12YDTGCE0VJU1HV4ZHBWAXPXD' # your Foursquare Secret
VERSION = '20180605'
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 12GJPABT1NNWQIJUMSESEY41PHZV0TPO1H3FRXSNZ3ZK0JMD
CLIENT_SECRET:FV4DBNONPOV1F3K3NPOGKXB12YDTGCE0VJU1HV4ZHBWAXPXD


#### Create function that extracts the category of the venue

In [359]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Explore Neighborhoods in Toronto

#### Create a function to repeat the same process to all the neighborhoods in Toronto

In [360]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called Toronto_venues

In [361]:
#Get Nearby venues
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

print("All Venues read....")

print(Toronto_venues.shape)
Toronto_venues.head()

All Venues read....
(1704, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [362]:
ITR = Toronto_venues[Toronto_venues['Venue Category'].str.contains('Restaurant') == True]
ITR.reset_index(inplace = True, drop=True)
Toronto_venues = ITR

In [363]:
#### Check how many venues and unique categories were returned for each neighborhood

In [364]:
print("Venues : \n") 
Toronto_venues.groupby('Neighborhood').count()

Venues : 



,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",27,27,27,27,27,27
Berczy Park,12,12,12,12,12,12
"Brockton, Exhibition Place, Parkdale Village",2,2,2,2,2,2
Business Reply Mail Processing Centre 969 Eastern,2,2,2,2,2,2
"Cabbagetown, St. James Town",11,11,11,11,11,11
Central Bay Street,23,23,23,23,23,23
"Chinatown, Grange Park, Kensington Market",34,34,34,34,34,34
Christie,2,2,2,2,2,2
Church and Wellesley,30,30,30,30,30,30


In [365]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 50 uniques categories.


## Analyze Each Neighborhood

In [366]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [367]:
Toronto_onehot.shape

(405, 51)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [368]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Dim Sum Restaurant,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,New American Restaurant,Persian Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,"Adelaide, King, Richmond",0.000000,0.148148,0.000000,0.111111,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.037037,0.000000,0.037037,0.000000,0.074074,0.00,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.111111,0.037037,0.000000,0.074074,0.000000,0.000000,0.148148,0.000000,0.037037,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.166667,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.181818,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.272727,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000
5,Central Bay Street,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,0.086957,0.173913,0.043478,0.00,0.043478,0.000000,0.000000,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.043478,0.043478,0.043478,0.043478,0.000000,0.043478,0.000000,0.000000,0.086957,0.000000,0.043478,0.000000
6,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.029412,0.000000,0.029412,0.000000,0.000000,0

#### Let's confirm the new size

In [369]:
Toronto_grouped.shape

(32, 51)

#### Let's print each neighborhood along with the top 5 most common venues

In [370]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0      Thai Restaurant  0.15
1  American Restaurant  0.15
2     Asian Restaurant  0.11
3           Restaurant  0.11
4     Sushi Restaurant  0.07


----Berczy Park----
                venue  freq
0          Restaurant  0.25
1  Italian Restaurant  0.17
2  Seafood Restaurant  0.17
3  Belgian Restaurant  0.08
4     Thai Restaurant  0.08


----Brockton, Exhibition Place, Parkdale Village----
                       venue  freq
0         Italian Restaurant   0.5
1       Caribbean Restaurant   0.5
2      Portuguese Restaurant   0.0
3          Korean Restaurant   0.0
4  Latin American Restaurant   0.0


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Restaurant   0.5
2     Afghan Restaurant   0.0
3    Persian Restaurant   0.0
4     Korean Restaurant   0.0


----Cabbagetown, St. James Town----
                  venue  freq
0            Restaurant  0.27


#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [371]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [372]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",American Restaurant,Thai Restaurant,Asian Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant,Indian Restaurant,New American Restaurant,Gluten-free Restaurant,Greek Restaurant
1,Berczy Park,Restaurant,Italian Restaurant,Seafood Restaurant,Comfort Food Restaurant,Thai Restaurant,French Restaurant,Belgian Restaurant,Greek Restaurant,Doner Restaurant,German Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Italian Restaurant,Caribbean Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
4,"Cabbagetown, St. James Town",Restaurant,Italian Restaurant,Indian Restaurant,Thai Restaurant,Taiwanese Restaurant,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Doner Restaurant,French Restaurant


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [373]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 4, 4, 4, 3, 3, 4, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [374]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#Remove all NA cases
Toronto_merged.dropna(inplace=True)

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2.0,Greek Restaurant,Italian Restaurant,Caribbean Restaurant,Restaurant,Indian Restaurant,American Restaurant,Belgian Restaurant,Eastern European Restaurant,Gluten-free Restaurant,German Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,4.0,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3.0,American Restaurant,Italian Restaurant,New American Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Latin American Restaurant,Chinese Restaurant,Comfort Food Restaurant,French Restaurant,Filipino Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1.0,Dim Sum Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0.0,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant


Finally, let's visualize the resulting clusters

In [375]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [427]:
## Define a method to show Specific Characteristics of the cluster
def Cluster_Details(Cluster, Filter, Criteria) :
    l1 = len(Cluster)
    l2 = 0
    Bors = Cluster['Borough']
    Cluster.drop(['Cluster Labels'], axis=1, inplace=True)
    if Cluster.empty == False :
        Cluster = Cluster[Cluster.apply(lambda row: row.astype(str).str.contains(Filter).any(), axis=1)]
        Cluster = Cluster.transpose()
        if Cluster.empty == False :
            #print(Cluster)
            Cluster = Cluster[Cluster.apply(lambda row: row.astype(str).str.contains(Filter).any(), axis=1)]
            Cluster = Cluster.transpose()
            l2 = len(Cluster)              
            Cluster = pd.merge(Cluster, Bors, left_index=True, right_index=True)
            
    print("Out of", l1, "Boroughs in the Cluster", l2, "Boroughs have", Filter, Criteria)  
    if l2 >0 :
        #print("Selected Bouroughs are:", Cluster['Borough'].to_string(index=False))
        print("Selected Bouroughs are: \n", Cluster['Borough'].value_counts().to_string())
    
    return Cluster

#### Cluster 1

In [428]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,Central Toronto,0.0,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
64,Central Toronto,0.0,Sushi Restaurant,Mexican Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
76,West Toronto,0.0,Fast Food Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
82,West Toronto,0.0,Mexican Restaurant,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Cajun / Creole Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant


In [429]:
Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()

Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')

Out of 4 Boroughs in the Cluster 1 Boroughs have Italian Restaurants
Selected Bouroughs are: 
 West Toronto    1


#### Cluster 2

In [430]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,1.0,Dim Sum Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [431]:
Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()

Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')

Out of 1 Boroughs in the Cluster 0 Boroughs have Italian Restaurants


#### Cluster 3

In [432]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,2.0,Greek Restaurant,Italian Restaurant,Caribbean Restaurant,Restaurant,Indian Restaurant,American Restaurant,Belgian Restaurant,Eastern European Restaurant,Gluten-free Restaurant,German Restaurant


In [433]:
Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()

Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')

Out of 1 Boroughs in the Cluster 1 Boroughs have Italian Restaurants
Selected Bouroughs are: 
 East Toronto    1


#### Cluster 4

In [434]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,East Toronto,3.0,American Restaurant,Italian Restaurant,New American Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Latin American Restaurant,Chinese Restaurant,Comfort Food Restaurant,French Restaurant,Filipino Restaurant
49,Central Toronto,3.0,Vietnamese Restaurant,American Restaurant,Sushi Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
52,Downtown Toronto,3.0,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Persian Restaurant,American Restaurant,Caribbean Restaurant,Chinese Restaurant
53,Downtown Toronto,3.0,Restaurant,Mexican Restaurant,Italian Restaurant,French Restaurant,Greek Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,Filipino Restaurant,Fast Food Restaurant
54,Downtown Toronto,3.0,Middle Eastern Restaurant,Thai Restaurant,Italian Restaurant,Japanese Restaurant,Restaurant,Ramen Restaurant,Vietnamese Restaurant,Chinese Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant
55,Downtown Toronto,3.0,Restaurant,Italian Restaurant,American Restaurant,Japanese Restaurant,Seafood Restaurant,Indian Restaurant,New American Restaurant,Korean Restaurant,Latin American Restaurant,Middle Eastern Restaurant
56,Downtown Toronto,3.0,Restaurant,Italian Restaurant,Seafood Restaurant,Comfort Food Restaurant,Thai Restaurant,French Restaurant,Belgian Restaurant,Greek Restaurant,Doner Restaurant,German Restaurant
57,Downtown Toronto,3.0,Italian Restaurant,Indian Restaurant,Thai Restaurant,Chinese Restaurant,Portuguese Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Korean Restaurant,Middle Eastern Restaurant
58,Downtown Toronto,3.0,American Restaurant,Thai Restaurant,Asian Restaurant,Restaurant,Sushi Restaurant,Japanese Restaurant,Indian Restaurant,New American Restaurant,Gluten-free Restaurant,Greek Restaurant
60,Downtown Toronto,3.0,Restaurant,American Restaurant,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,New American Restaurant,Fast Food Restaurant,Gluten-free Restaurant,French Restaurant,Asian Restaurant


In [435]:
Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()

Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')

Out of 16 Boroughs in the Cluster 10 Boroughs have Italian Restaurants
Selected Bouroughs are: 
 Downtown Toronto    8
West Toronto        1
East Toronto        1


#### Cluster 5

In [436]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,4.0,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
47,Central Toronto,4.0,Sushi Restaurant,Italian Restaurant,Restaurant,Indian Restaurant,Japanese Restaurant,Seafood Restaurant,Greek Restaurant,Thai Restaurant,Belgian Restaurant,Dumpling Restaurant
51,Downtown Toronto,4.0,Restaurant,Italian Restaurant,Indian Restaurant,Thai Restaurant,Taiwanese Restaurant,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Doner Restaurant,French Restaurant
59,Downtown Toronto,4.0,Italian Restaurant,Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Seafood Restaurant,Chinese Restaurant,New American Restaurant
66,Downtown Toronto,4.0,Japanese Restaurant,Restaurant,Italian Restaurant,French Restaurant,Sushi Restaurant,Chinese Restaurant,Comfort Food Restaurant,Vietnamese Restaurant,Dumpling Restaurant,German Restaurant
75,Downtown Toronto,4.0,Italian Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
78,West Toronto,4.0,Italian Restaurant,Caribbean Restaurant,Doner Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
83,West Toronto,4.0,Italian Restaurant,Restaurant,Eastern European Restaurant,Cuban Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant
84,West Toronto,4.0,Italian Restaurant,Sushi Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Latin American Restaurant,Restaurant,Falafel Restaurant,Vietnamese Restaurant,Doner Restaurant,Filipino Restaurant
87,East Toronto,4.0,Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [437]:
Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()

Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')

Out of 10 Boroughs in the Cluster 9 Boroughs have Italian Restaurants
Selected Bouroughs are: 
 Downtown Toronto    4
West Toronto        3
Central Toronto     1
East Toronto        1


In [ ]:
For n in range(5):
    Cluster = Toronto_merged.loc[Toronto_merged['Cluster Labels'] == n, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]].copy()
    Cluster = Cluster_Details(Cluster, 'Italian', 'Restaurants')